# Develop -- Launch BLVM Pipeline

In [2]:
import boto3
import sagemaker
from datetime import date
from sagemaker.workflow.parallelism_config import ParallelismConfiguration

import utils

In [3]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()
# default_bucket = "iberia-data-lake"
environment = "develop"
environment_ppl = "prod" if environment == "production" else "sbx"
configuration = utils.read_config_data()
str_execution_date = str(date(date.today().year, date.today().month, date.today().day - 5))
print(f"str_execution_date={str_execution_date}")
is_last_date = "0"
model_version = configuration.get("MODEL_VERSION")
vertical = configuration.get("VERTICAL")
s3_path = f"{vertical}/{environment_ppl}/{model_version.lower()}"
s3_path_read = "customer/customer_models_data"
model_type = "bl"
use_type = "train"

str_execution_date=2022-12-23


## Pipeline definition

In [4]:
from production.pipelines_code.pipeline import get_pipeline as pipeline_definition

pipeline_name = (
    f"ibdata-{vertical}-{model_version.lower()}-{environment_ppl}-ppl"
)
base_job_prefix = pipeline_name

pipe_definition = pipeline_definition(
    region=region,
    role=role,
    default_bucket=default_bucket,
    pipeline_name=pipeline_name,
    base_job_prefix=base_job_prefix
)
# all_step.upsert(role_arn=role)

This function has been deprecated and could break pipeline step caching. We recommend using the run() function directly with pipeline sessionsto access step arguments.


In [5]:
# This cell starts an execution of the previous pipeline created
execution = pipe_definition.start(
    parallelism_config=ParallelismConfiguration(max_parallel_execution_steps=4).to_request(),
    parameters=dict(
        str_execution_date=str_execution_date,
        is_last_date=is_last_date,
        s3_path=s3_path,
        s3_path_read=s3_path_read,
        model_type=model_type,
        use_type=use_type
    )
)
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-blv-sbx-ppl',
 'PipelineExecutionArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-blv-sbx-ppl/execution/xiz1fh32n9pz',
 'PipelineExecutionDisplayName': 'execution-1672240944345',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 12, 28, 15, 22, 24, 258000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 12, 28, 15, 22, 24, 258000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:077156906314:user-profile/d-8rnxtw3g36nb/customer-esteban-sanchez',
  'UserProfileName': 'customer-esteban-sanchez',
  'DomainId': 'd-8rnxtw3g36nb'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:077156906314:user-profile/d-8rnxtw3g36nb/customer-esteban-sanchez',
  'UserProfileName': 'customer-esteban-sanchez',
  'DomainId': 'd-8rnxtw3g36nb'},
 'ParallelismConfiguration': {'MaxParallelExecutionSteps': 4},
 